<a href="https://colab.research.google.com/github/crisprking/miniprotein_genai/blob/main/notebooks/01_backbone_RFdiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<details><summary>Cell 1 – Install packages (run once per session)</summary>
python

#@title 🔧 Install RFdiffusion & helpers (~3 min)
%%bash
python - <<'PY'
import subprocess, sys
pkgs = [
    "git+https://github.com/sokrypton/RFdiffusion@v1.1.1",
    "git+https://github.com/sokrypton/ColabDesign@v1.1.1",
    "py3Dmol", "omegaconf", "hydra-core"
]
subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
print("✅ Packages installed")
PY

</details> <details><summary>Cell 2 – Design parameters</summary>
python
Copy

#@title ⚙️ Parameters
name        = "design1"  #@param {type:"string"}
length      = 100        #@param {type:"integer"}
symmetry    = "none"     #@param ["none","cyclic","dihedral"]
order       = 1          #@param {type:"integer"}
num_designs = 1          #@param {type:"integer"}
iterations  = 50         #@param ["25","50","100","150","200"] {type:"raw"}
print(f"Backbone length {length}, copies {num_designs}")

</details> <details><summary>Cell 3 – Run RFdiffusion</summary>
python
Copy

#@title 🚀 Run RFdiffusion
import subprocess, pathlib, sys
contigs = f"{length}"
out_prefix = f"outputs/{name}"

cmd = [
    "python","-m","RFdiffusion.run_inference",
    f"contigmap.contigs=[{contigs}]",
    f"inference.output_prefix={out_prefix}",
    f"inference.num_designs={num_designs}",
    f"diffuser.T={iterations}",
    "inference.dump_pdb=True",
    "inference.dump_pdb_path=/dev/shm"
]
if symmetry != "none":
    symflag = f"{symmetry[0]}{order}"
    cmd = ["python","-m","RFdiffusion.run_inference","--config-name","symmetry",
           f"inference.symmetry={symflag}"] + cmd[2:]

subprocess.check_call(cmd)
print("✅ Backbone(s) finished; files in /content/outputs/")

</details> <details><summary>Cell 4 – Interactive preview</summary>
python
Copy

#@title 👀 Preview first backbone
import glob, py3Dmol, pathlib
pdb = glob.glob(f"outputs/{name}_0.pdb")[0]
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(open(pdb).read(),'pdb')
view.setStyle({'cartoon':{'color':'spectrum'}})
view.zoomTo(); view.show()

</details> <details><summary>Cell 5 – Zip download</summary>
python
Copy

#@title 📦 Download result
import shutil, google.colab
zip_name = f"{name}.result"
shutil.make_archive(zip_name, "zip", root_dir="outputs", base_dir=name)
google.colab.files.download(f"{zip_name}.zip")

